<a href="https://colab.research.google.com/github/ydydydydydy/Colab/blob/main/ex102_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# input = 가는
# output = 말이 고와야 오는 말이 곱다
# 첫마디 입력시 뒤에 나올 말 생

In [1]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다"""
# 바로 전의 말만 확인하고 텍스트를 생성하는 게 아니라 전에 나왔던 모든 말을 확인하고
# 텍스트를 생성하기 때문에 '말이' 다음에 나오는 다양한 말들을 생성 가능

In [3]:
# 띄어쓰기를 기준으로 단어 분리
from tensorflow.keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts([text])

t.word_index

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [5]:
# 훈련 데이터 생성
sequences = []
for line in text.split('\n'):
  # 단어를 정수로 변경
  encoded = t.texts_to_sequences([line])[0]

 # 2개 이상의 조합 생성
  for i in range(1,len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

In [6]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 6

sequences = pad_sequences(sequences, maxlen = max_len)
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]], dtype=int32)

In [8]:
import numpy as np
sequences = np.array(sequences)
X = sequences[:,:-1] # 문제 데이터 앞에 5 단어 // (array [0, 0, 0, 0, 2 ])
y = sequences[:,-1] # 정답 데이터 뒤에 1 단어 // (array [3])

In [9]:
# 원핫인코딩
from tensorflow.keras.utils import to_categorical

y_en = to_categorical(y,num_classes = 12)
y_en

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from tensorflow.keras import Sequential

model1 = Sequential()

model1.add(Embedding(12,10,input_length = 5))
model1.add(SimpleRNN(32))
model1.add(Dense(units = 12, activation = 'softmax'))

model1.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

model1.fit(X, y_en, epochs = 200)

In [12]:
# 학습 모델 사용해서 텍스트 생성기능 만들기
# 사용할 모델, 토크나이저, 입력 단어, 생성할 단어 수
def make_sentence(model, t, current_word, n):
  init_word = current_word # 마지막에 같이 출력하기 위함
  sentence = ''
  for _ in range(n):
    encoded = t.texts_to_sequences([current_word])[0]
    print(encoded) # 번호 확인
    print(current_word) # 입력 단어 확인
    # 데이터를 같은 길이(5)로 맞춰주기
    encoded = pad_sequences([encoded], maxlen = max_len - 1)
    # 입력한 단어로 예측하기
    result = model.predict(encoded, verbose = 0).argmax(axis= -1)
    for word, index in t.word_index.items():
      if index == result:
        break

    # 현재단어 + " " + 예측단어를 현재 단어로 변경
    current_word = current_word + " " + word

    sentence = sentence + " " + word
  # 문장 생성
  sentence = init_word + sentence

  return sentence

In [14]:
# 인공지능 딥러닝 모델, 토큰화 모델, 시작 단어
# 경마장에 있는 말이 뛰고 있다
# 그의 말이 법이다
# 가는 말이 고와야 오는 말이 곱다
print(make_sentence(model1, t, '경마장에', 4)) # 출력해야 되는 말이 4개

[2]
경마장에
[2, 3]
경마장에 있는
[2, 3, 1]
경마장에 있는 말이
[2, 3, 1, 4]
경마장에 있는 말이 뛰고
경마장에 있는 말이 뛰고 있다


In [15]:
print(make_sentence(model1, t, '그의', 2))

[6]
그의
[6, 1]
그의 말이
그의 말이 법이다


In [16]:
print(make_sentence(model1, t, '가는', 5))

[8]
가는
[8, 1]
가는 말이
[8, 1, 9]
가는 말이 고와야
[8, 1, 9, 10]
가는 말이 고와야 오는
[8, 1, 9, 10, 1]
가는 말이 고와야 오는 말이
가는 말이 고와야 오는 말이 곱다
